In [ ]:
#importing libaries and initialising my network
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import plotly.graph_objects as go
#from plotly.colors import n_colorsy
import Deep_Neural_Network_Script
import mpmath

torch.cuda.empty_cache()

npts = 1

device = torch.device("cpu")


poly_model = Deep_Neural_Network_Script.NeuralNetwork(
    number_of_hidden_layers= 10, 
    number_of_auxillary_variable = 3,
    number_of_phase_space_parameters = 3,
    hidden_layer_size = 30,
    output_layer_size = 1,
    activation_function = "GELU",
    batch_size = 500,
    normalisation_coefficient = 120,
    xavier_gain = 1.5
) 
torch.set_printoptions(precision=20)

normalisation_coefficient = poly_model.normalisation_coefficient

saved_model = '1L_GELU_10layer_30hidden_branch1_run9.pth'

In [ ]:
#do some sort of normalisation for the features from the kaggle set. 
def s_normalisation(input_nodes):
    if poly_model.activation_function == 'newGELU':  #or 'GELU
        return (input_nodes + 30)/27  #makes all in puts between 0 and 1
    else:
        return input_nodes/30# + 29.99)/27
   



#training tools
loss_function = nn.MSELoss()
optimizer = optim.Adam(poly_model.parameters(), lr =0.01)#
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.99, patience = 60, verbose=True)# 0.75


network_gradients_array_hidden = []
network_gradients_array_input = []
network_gradients_array_output = []
network_gradients_array_hidden_std = []
network_gradients_array_input_std = []
network_gradients_array_output_std = []


iterations = 1000 
freq = 10
npts_cycle = int(iterations/(freq))

#validation_set_size = 

#arrays to store information from the training
y_max_array = [] 
y_min_array = []
y_mean_array = []
y_std_array = []
loss_array = np.zeros(int(freq))
val_loss_array = np.zeros(int(freq))
iteration_array = np.zeros(int(freq))
lr_array = np.zeros(int(freq))

#make these np.zero arrays and fix values to them to increase speed
activation_value_array = [] #6 is because number of hidden layers + 2
activation_value_array_std = [] #6 is because number of hidden layers + 2
weight_array = []
bias_array = []

#training loop for regression using y_observed and y_predicted 
for epoch in range(iterations):

    if epoch == 10 or epoch ==int(iterations)-1:
        with torch.no_grad():
            network_gradients_array_hidden.append(poly_model.weights_hidden_hidden.grad)
            network_gradients_array_input.append(poly_model.weights_input_hidden.grad)
            network_gradients_array_output.append(poly_model.weights_hidden_output.grad)

   #zero gradients
    optimizer.zero_grad()   
      

    y_observed = ("result of match from Kaggle") #from featurs off of Kaggle
   
    y_predicted = poly_model.forward(normalised("feature off two sets of fighter off Kaggle")) #outputs from my network

    
    loss = loss_function(y_predicted, y_observed)
    
    loss.backward() #    #backward pass


#    torch.nn.utils.clip_grad_norm_(poly_model.parameters(), max_norm= 5e-5)
   # for name, param in poly_model.named_parameters():
    #    if param.grad is not None:
     #       print(f'Gradient of {name}:\n{param.grad}')
      #  else:
       #     print(f'No gradient for {name}')

    optimizer.step() #update parameters
    
    
    #check backprop   
   # print(f'Parameter: poly_model.weights_hidden_output, Gradient: {poly_model.weights_hidden_output.grad[93:97]}')
    #print(f'Parameter: poly_model.weights_hidden_hidden, Gradient: {poly_model.weights_hidden_hidden.grad[:,97, 19:23]}')
    #print(f'Parameter: poly_model.weights_input_hidden, Gradient: {poly_model.weights_input_hidden.grad[:, 1:4]}')
   # print(f'Parameter: poly_model.bias_hidden, Gradient: {poly_model.bias_hidden.grad[0:3, 0:3]}')
    #for name, param in poly_model.named_parameters():
        #print(f'gradient: {param.grad.shape}')
        #print(f'Parameter: {name}, Gradient: {param.grad[0][0:6]}')
        #print(f'Parameter: {name}, Gradient: {param.grad[0][0:6]}')
        #Parameter: weights_input_hidden, Gradient: torch.Size([6, 100])
        #Parameter: weights_hidden_hidden, Gradient: torch.Size([2, 100, 100])
        #Parameter: weights_hidden_output, Gradient: torch.Size([100, 1])
        #Parameter: bias_hidden, Gradient: torch.Size([4, 100])#
  
    #validation set 
    poly_model.eval() #
    with torch.no_grad():
        y_observed_val = ("result of match from Kaggle (validation set)") #from featurs off of Kaggle
   
        y_predicted_val = poly_model.forward(normalised("feature off two sets of fighter off Kaggle (validation set)")) #outputs from my network

        val_loss = loss_function(y_observed_val, y_predicted_val).to(device)
        #print(val_loss) 

    scheduler.step(val_loss)
    
    
    poly_model.train()
   
    
    #figure out the collection rate for network performance params. 

    if (epoch + 1) % collection_rate == 0: #10000   
        poly_model.eval()
        with torch.no_grad():
      #      print(f'network integrand: {outputs[0]}')
       #     print(f'true integrand: {y}')
            loss_array[i_indication] = loss
            val_loss_array[i_indication] = val_loss
            iteration_array[i_indication] = epoch + 1
            lr_array[i_indication] = optimizer.param_groups[0]['lr']
     
        poly_model.train()
   
  
